In [3]:
import sys
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/poc")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/plugins")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

#Import libs python
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date

#Import libs internas
from utils import spark_utils_session as utils

from hooks.hdfs.hdfs_helper import HdfsHelper
from jobs.job_base_config import BaseETLJobClass

import poc_helper
poc_helper.load_env("PROD")

'.env_file' loaded!
ENV 'PROD' configured!


In [4]:
def get_session(profile: str, dynamic_allocation_enabled: bool = True) -> utils.DBASparkAppSession:
    """Generates DBASparkAppSession."""
    
    app_name = "tsevero_cred_sql_validar"
    
    spark_builder = (utils.DBASparkAppSession
                     .builder
                     .setAppName(app_name)
                     .usingProcessProfile(profile)
                    )
    
    if dynamic_allocation_enabled:
        spark_builder.autoResourceManagement()

    return spark_builder.build()

session = get_session(profile='efd_t2')

2025-11-27T17:56:42.809697Z [info     ] Using json file settings.      [root] loc=spark_utils_session.py:301
2025-11-27T17:56:42.810883Z [info     ] Exporting default ENV.         [root] loc=spark_utils_session.py:305
2025-11-27T17:56:42.811410Z [info     ] Exporting custom ENVs.         [root] loc=spark_utils_session.py:338
2025-11-27T17:56:42.811955Z [info     ] Building profile 'efd_t2'.     [root] loc=spark_utils_session.py:221
2025-11-27T17:56:42.812301Z [info     ] Not enough info for building the kerberos client. Ignoring it [root] loc=spark_utils_session.py:284


:: loading settings :: url = jar:file:/opt/cloudera/parcels/SPARK3-3.5.4.3.5.7191000.0-30-1.p0.68499982/lib/spark3/jars/ivy-2.5.2.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/tsevero/.ivy2/cache
The jars for the packages stored in: /home/tsevero/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
com.oracle.database.security#oraclepki added as a dependency
com.oracle.database.security#osdt_core added as a dependency
com.oracle.database.security#osdt_cert added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-864078dc-2c00-4353-bd95-63346bcb5cfd;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.18.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.2 in central
	found com.oracle.database.security#oraclepki;21.19.0.0 in central
	found com.oracle.databa

In [5]:
session.sparkSession.sql("SHOW DATABASES").show(truncate=False)

Hive Session ID = 6b66bcb2-e1f8-445a-9ebd-76302b1e6fa7


+------------------+
|namespace         |
+------------------+
|anac              |
|bcadastro         |
|bpe               |
|c115              |
|ccc               |
|ccg               |
|cte               |
|default           |
|destda            |
|detran_share      |
|dime              |
|due               |
|efd               |
|fci               |
|gecob             |
|gescol            |
|gessimples        |
|gplam             |
|information_schema|
|malhas            |
+------------------+
only showing top 20 rows



In [6]:
# ============================================================================
# CONFIGURAÇÃO INICIAL - 
# ============================================================================

import sys
import warnings
from datetime import datetime, date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PySpark imports com aliases para evitar conflitos
from pyspark.sql.functions import (
    col as spark_col, 
    sum as spark_sum, 
    avg as spark_avg,
    count as spark_count,
    when as spark_when,
    desc as spark_desc,
    asc as spark_asc,
    round as spark_round,
    concat as spark_concat,
    lit as spark_lit,
    max as spark_max,
    min as spark_min,
    stddev as spark_stddev,
    countDistinct as spark_countDistinct
)
from pyspark.sql.types import DoubleType, IntegerType

# Configurações de visualização
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 11

# ✅ CORREÇÃO: Não usar abs() que conflita com PySpark
# pd.set_option('display.float_format', lambda x: f'{x:,.2f}' if abs(x) > 0.01 else f'{x:.6f}')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Acesso ao Spark
spark = session.sparkSession

print("=" * 80)
print("🔍 SISTEMA")
print("=" * 80)
print(f"Sessão Spark: {spark.sparkContext.appName}")
print(f"Versão Spark: {spark.version}")
print(f"Iniciado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)

🔍 SISTEMA
Sessão Spark: tsevero_cred_sql_validar
Versão Spark: 3.5.4.3.5.7191000.0-30
Iniciado em: 2025-11-27 14:57:12


In [7]:
"""
DIAGNÓSTICO COMPLETO: DECLARAÇÕES ZERADAS E OMISSÕES
Baseado nas tabelas: cancel_zero_normal, cancel_zero_simples e credito_dime_completo
DIME (Regime Normal) vs PGDAS (Simples Nacional)
"""

print("="*80)
print("DIAGNÓSTICO: DECLARAÇÕES ZERADAS E OMISSÕES")
print("DIME (Regime Normal) vs PGDAS (Simples Nacional)")
print("="*80)

# ============================================================================
# 1. VERIFICAR ESTRUTURA DAS TABELAS
# ============================================================================
print("\n" + "="*60)
print("1. ESTRUTURA DAS TABELAS")
print("="*60)

tabelas = {
    'teste.cancel_zero_normal': 'Regime Normal (DIME)',
    'teste.cancel_zero_simples': 'Simples Nacional (PGDAS)',
    'teste.credito_dime_completo': 'Tabela Principal Consolidada'
}

for tabela, descricao in tabelas.items():
    try:
        print(f"\n📋 {descricao} ({tabela}):")
        colunas = spark.sql(f"DESCRIBE {tabela}").collect()
        
        # Filtrar colunas relevantes
        colunas_rel = [col.col_name for col in colunas if any(x in col.col_name.lower() 
                       for x in ['periodo', 'omiss', 'zerad', 'flag', 'indicio'])]
        
        if colunas_rel:
            for col in colunas_rel:
                print(f"   ✓ {col}")
        else:
            print(f"   ⚠️ Nenhuma coluna de declaração encontrada")
            
        # Contar total de registros
        total = spark.sql(f"SELECT COUNT(*) as total FROM {tabela}").collect()[0]['total']
        print(f"   📊 Total de registros: {total:,}")
        
    except Exception as e:
        print(f"   ❌ Erro ao acessar tabela: {str(e)[:100]}")

# ============================================================================
# 2. ANÁLISE DETALHADA: cancel_zero_normal (REGIME NORMAL - DIME)
# ============================================================================
print("\n" + "="*60)
print("2. ANÁLISE: cancel_zero_normal (REGIME NORMAL - DIME)")
print("="*60)

try:
    stats_normal = spark.sql("""
        SELECT 
            COUNT(*) as total_registros,
            COUNT(DISTINCT nu_cnpj) as total_cnpjs,
            
            -- Períodos Omissos e Zerados
            SUM(CASE WHEN periodos_omissos > 0 THEN 1 ELSE 0 END) as com_omissos,
            SUM(CASE WHEN periodos_zerados > 0 THEN 1 ELSE 0 END) as com_zerados,
            SUM(CASE WHEN periodos_sem_movimento > 0 THEN 1 ELSE 0 END) as com_sem_movimento,
            
            ROUND(AVG(periodos_omissos), 2) as media_omissos,
            ROUND(AVG(periodos_zerados), 2) as media_zerados,
            ROUND(AVG(periodos_sem_movimento), 2) as media_sem_movimento,
            
            MAX(periodos_omissos) as max_omissos,
            MAX(periodos_zerados) as max_zerados,
            
            -- Indícios de Fraude
            SUM(CASE WHEN qtde_indicios_fraude > 0 THEN 1 ELSE 0 END) as com_indicios,
            SUM(CASE WHEN flag_omissao_dime_indicio = 1 THEN 1 ELSE 0 END) as flag_dime,
            SUM(CASE WHEN flag_omissao_pgdas_indicio = 1 THEN 1 ELSE 0 END) as flag_pgdas,
            
            ROUND(AVG(qtde_indicios_fraude), 2) as media_indicios,
            ROUND(AVG(soma_score_indicios), 2) as media_score_indicios
        FROM teste.cancel_zero_normal
    """).collect()[0]
    
    print(f"\n📊 Resumo Geral:")
    print(f"   Total de registros: {stats_normal['total_registros']:,}")
    print(f"   Total de CNPJs únicos: {stats_normal['total_cnpjs']:,}")
    
    print(f"\n📋 Períodos:")
    print(f"   Com períodos omissos: {stats_normal['com_omissos']:,} ({stats_normal['com_omissos']/stats_normal['total_registros']*100:.1f}%)")
    print(f"   Com períodos zerados: {stats_normal['com_zerados']:,} ({stats_normal['com_zerados']/stats_normal['total_registros']*100:.1f}%)")
    print(f"   Com sem movimento: {stats_normal['com_sem_movimento']:,} ({stats_normal['com_sem_movimento']/stats_normal['total_registros']*100:.1f}%)")
    print(f"   Média omissos: {stats_normal['media_omissos']:.2f}")
    print(f"   Média zerados: {stats_normal['media_zerados']:.2f}")
    print(f"   Máximo omissos: {stats_normal['max_omissos']:,}")
    print(f"   Máximo zerados: {stats_normal['max_zerados']:,}")
    
    print(f"\n🔍 Indícios:")
    print(f"   Com indícios de fraude: {stats_normal['com_indicios']:,}")
    print(f"   Flag omissão DIME: {stats_normal['flag_dime']:,}")
    print(f"   Flag omissão PGDAS: {stats_normal['flag_pgdas']:,} ⚠️ (DEVERIA SER 0 - NORMAL NÃO TEM PGDAS)")
    print(f"   Média de indícios: {stats_normal['media_indicios']:.2f}")
    print(f"   Média score: {stats_normal['media_score_indicios']:.2f}")
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 3. ANÁLISE DETALHADA: cancel_zero_simples (SIMPLES NACIONAL - PGDAS)
# ============================================================================
print("\n" + "="*60)
print("3. ANÁLISE: cancel_zero_simples (SIMPLES NACIONAL - PGDAS)")
print("="*60)

try:
    stats_simples = spark.sql("""
        SELECT 
            COUNT(*) as total_registros,
            COUNT(DISTINCT nu_cnpj) as total_cnpjs,
            
            -- Períodos Omissos e Zerados
            SUM(CASE WHEN periodos_omissos > 0 THEN 1 ELSE 0 END) as com_omissos,
            SUM(CASE WHEN periodos_zerados > 0 THEN 1 ELSE 0 END) as com_zerados,
            
            ROUND(AVG(periodos_omissos), 2) as media_omissos,
            ROUND(AVG(periodos_zerados), 2) as media_zerados,
            
            MAX(periodos_omissos) as max_omissos,
            MAX(periodos_zerados) as max_zerados,
            
            -- Indícios de Fraude
            SUM(CASE WHEN qtde_indicios_fraude > 0 THEN 1 ELSE 0 END) as com_indicios,
            SUM(CASE WHEN flag_omissao_dime_indicio = 1 THEN 1 ELSE 0 END) as flag_dime,
            SUM(CASE WHEN flag_omissao_pgdas_indicio = 1 THEN 1 ELSE 0 END) as flag_pgdas,
            
            ROUND(AVG(qtde_indicios_fraude), 2) as media_indicios,
            ROUND(AVG(soma_score_indicios), 2) as media_score_indicios
        FROM teste.cancel_zero_simples
    """).collect()[0]
    
    print(f"\n📊 Resumo Geral:")
    print(f"   Total de registros: {stats_simples['total_registros']:,}")
    print(f"   Total de CNPJs únicos: {stats_simples['total_cnpjs']:,}")
    
    print(f"\n📋 Períodos:")
    print(f"   Com períodos omissos: {stats_simples['com_omissos']:,} ({stats_simples['com_omissos']/stats_simples['total_registros']*100:.1f}%)")
    print(f"   Com períodos zerados: {stats_simples['com_zerados']:,} ({stats_simples['com_zerados']/stats_simples['total_registros']*100:.1f}%)")
    print(f"   Média omissos: {stats_simples['media_omissos']:.2f}")
    print(f"   Média zerados: {stats_simples['media_zerados']:.2f}")
    print(f"   Máximo omissos: {stats_simples['max_omissos']:,}")
    print(f"   Máximo zerados: {stats_simples['max_zerados']:,}")
    
    print(f"\n🔍 Indícios:")
    print(f"   Com indícios de fraude: {stats_simples['com_indicios']:,}")
    print(f"   Flag omissão DIME: {stats_simples['flag_dime']:,} ⚠️ (DEVERIA SER 0 - SIMPLES NÃO TEM DIME)")
    print(f"   Flag omissão PGDAS: {stats_simples['flag_pgdas']:,}")
    print(f"   Média de indícios: {stats_simples['media_indicios']:.2f}")
    print(f"   Média score: {stats_simples['media_score_indicios']:.2f}")
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 4. ANÁLISE: credito_dime_completo (TABELA PRINCIPAL)
# ============================================================================
print("\n" + "="*60)
print("4. ANÁLISE: credito_dime_completo (TABELA PRINCIPAL)")
print("="*60)

try:
    stats_completo = spark.sql("""
        SELECT 
            COUNT(*) as total_empresas,
            
            -- Flags
            SUM(CASE WHEN flag_omissao_dime_normal = 1 THEN 1 ELSE 0 END) as flag_omissao_dime_normal,
            SUM(CASE WHEN flag_omissao_dime_simples = 1 THEN 1 ELSE 0 END) as flag_omissao_dime_simples,
            SUM(CASE WHEN flag_omissao_pgdas_normal = 1 THEN 1 ELSE 0 END) as flag_omissao_pgdas_normal,
            SUM(CASE WHEN flag_omissao_pgdas_simples = 1 THEN 1 ELSE 0 END) as flag_omissao_pgdas_simples,
            
            -- Períodos Normal
            SUM(CASE WHEN periodos_zerados_normal > 0 THEN 1 ELSE 0 END) as com_zerados_normal,
            SUM(CASE WHEN periodos_omissos_normal > 0 THEN 1 ELSE 0 END) as com_omissos_normal,
            ROUND(AVG(periodos_zerados_normal), 2) as media_zerados_normal,
            ROUND(AVG(periodos_omissos_normal), 2) as media_omissos_normal,
            
            -- Períodos Simples
            SUM(CASE WHEN periodos_zerados_simples > 0 THEN 1 ELSE 0 END) as com_zerados_simples,
            SUM(CASE WHEN periodos_omissos_simples > 0 THEN 1 ELSE 0 END) as com_omissos_simples,
            ROUND(AVG(periodos_zerados_simples), 2) as media_zerados_simples,
            ROUND(AVG(periodos_omissos_simples), 2) as media_omissos_simples,
            
            -- Indícios
            ROUND(AVG(qtde_indicios_normal), 2) as media_indicios_normal,
            ROUND(AVG(qtde_indicios_simples), 2) as media_indicios_simples,
            ROUND(AVG(soma_score_indicios_normal), 2) as media_score_normal,
            ROUND(AVG(soma_score_indicios_simples), 2) as media_score_simples
        FROM teste.credito_dime_completo
    """).collect()[0]
    
    print(f"\n📊 Total de empresas: {stats_completo['total_empresas']:,}")
    
    print(f"\n🔴 FLAGS DE OMISSÃO:")
    print(f"   flag_omissao_dime_normal: {stats_completo['flag_omissao_dime_normal']:,} ✅ (CORRETO)")
    print(f"   flag_omissao_dime_simples: {stats_completo['flag_omissao_dime_simples']:,} ⚠️ (INCORRETO - DIME NÃO TEM SIMPLES)")
    print(f"   flag_omissao_pgdas_normal: {stats_completo['flag_omissao_pgdas_normal']:,} ⚠️ (INCORRETO - PGDAS NÃO TEM NORMAL)")
    print(f"   flag_omissao_pgdas_simples: {stats_completo['flag_omissao_pgdas_simples']:,} ✅ (CORRETO)")
    
    print(f"\n📋 PERÍODOS - REGIME NORMAL:")
    print(f"   Com zerados: {stats_completo['com_zerados_normal']:,} (média: {stats_completo['media_zerados_normal']:.2f})")
    print(f"   Com omissos: {stats_completo['com_omissos_normal']:,} (média: {stats_completo['media_omissos_normal']:.2f})")
    
    print(f"\n📋 PERÍODOS - SIMPLES NACIONAL:")
    print(f"   Com zerados: {stats_completo['com_zerados_simples']:,} (média: {stats_completo['media_zerados_simples']:.2f})")
    print(f"   Com omissos: {stats_completo['com_omissos_simples']:,} (média: {stats_completo['media_omissos_simples']:.2f})")
    
    print(f"\n🔍 INDÍCIOS:")
    print(f"   Média indícios Normal: {stats_completo['media_indicios_normal']:.2f} (score: {stats_completo['media_score_normal']:.2f})")
    print(f"   Média indícios Simples: {stats_completo['media_indicios_simples']:.2f} (score: {stats_completo['media_score_simples']:.2f})")
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 5. CRUZAMENTO DE DADOS: ORIGEM vs DESTINO
# ============================================================================
print("\n" + "="*60)
print("5. CRUZAMENTO: TABELAS DE ORIGEM vs DESTINO")
print("="*60)

try:
    cruzamento = spark.sql("""
        WITH origem_normal AS (
            SELECT DISTINCT REGEXP_REPLACE(TRIM(CAST(nu_cnpj AS STRING)), '[^0-9]', '') AS cnpj_limpo
            FROM teste.cancel_zero_normal
            WHERE periodos_zerados > 0 OR periodos_omissos > 0
        ),
        origem_simples AS (
            SELECT DISTINCT REGEXP_REPLACE(TRIM(CAST(nu_cnpj AS STRING)), '[^0-9]', '') AS cnpj_limpo
            FROM teste.cancel_zero_simples
            WHERE periodos_zerados > 0 OR periodos_omissos > 0
        ),
        destino AS (
            SELECT DISTINCT REGEXP_REPLACE(TRIM(CAST(nu_cnpj AS STRING)), '[^0-9]', '') AS cnpj_limpo
            FROM teste.credito_dime_completo
            WHERE periodos_zerados_normal > 0 
               OR periodos_zerados_simples > 0
               OR periodos_omissos_normal > 0
               OR periodos_omissos_simples > 0
        )
        SELECT 
            (SELECT COUNT(*) FROM origem_normal) as origem_normal_cnpjs,
            (SELECT COUNT(*) FROM origem_simples) as origem_simples_cnpjs,
            (SELECT COUNT(*) FROM destino) as destino_cnpjs,
            (SELECT COUNT(*) FROM origem_normal o WHERE EXISTS (SELECT 1 FROM destino d WHERE d.cnpj_limpo = o.cnpj_limpo)) as match_normal,
            (SELECT COUNT(*) FROM origem_simples o WHERE EXISTS (SELECT 1 FROM destino d WHERE d.cnpj_limpo = o.cnpj_limpo)) as match_simples
    """).collect()[0]
    
    print(f"\n📊 CNPJs nas tabelas de origem:")
    print(f"   cancel_zero_normal (com declarações): {cruzamento['origem_normal_cnpjs']:,}")
    print(f"   cancel_zero_simples (com declarações): {cruzamento['origem_simples_cnpjs']:,}")
    
    print(f"\n📊 CNPJs na tabela destino:")
    print(f"   credito_dime_completo (com declarações): {cruzamento['destino_cnpjs']:,}")
    
    if cruzamento['origem_normal_cnpjs'] > 0:
        perc_normal = cruzamento['match_normal'] / cruzamento['origem_normal_cnpjs'] * 100
        print(f"\n✅ Matches Normal:")
        print(f"   {cruzamento['match_normal']:,} / {cruzamento['origem_normal_cnpjs']:,} ({perc_normal:.1f}%)")
    
    if cruzamento['origem_simples_cnpjs'] > 0:
        perc_simples = cruzamento['match_simples'] / cruzamento['origem_simples_cnpjs'] * 100
        print(f"\n✅ Matches Simples:")
        print(f"   {cruzamento['match_simples']:,} / {cruzamento['origem_simples_cnpjs']:,} ({perc_simples:.1f}%)")
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 6. IDENTIFICAR INCONSISTÊNCIAS
# ============================================================================
print("\n" + "="*60)
print("6. IDENTIFICAR INCONSISTÊNCIAS")
print("="*60)

print("\n🔍 Verificando casos problemáticos:")

# Inconsistência 1: Flag mas sem períodos
try:
    incon1 = spark.sql("""
        SELECT COUNT(*) as qtd
        FROM teste.credito_dime_completo
        WHERE flag_omissao_dime_normal = 1 
          AND (periodos_omissos_normal IS NULL OR periodos_omissos_normal = 0)
    """).collect()[0]['qtd']
    
    if incon1 > 0:
        print(f"   ⚠️ Inconsistência 1: {incon1:,} empresas com flag_omissao_dime_normal=1 mas periodos_omissos_normal=0")
    else:
        print(f"   ✅ OK: Flag omissão DIME Normal consistente com períodos omissos")
except:
    print(f"   ⚠️ Não foi possível verificar inconsistência 1")

# Inconsistência 2: Períodos mas sem flag
try:
    incon2 = spark.sql("""
        SELECT COUNT(*) as qtd
        FROM teste.credito_dime_completo
        WHERE periodos_omissos_normal > 0
          AND (flag_omissao_dime_normal IS NULL OR flag_omissao_dime_normal = 0)
    """).collect()[0]['qtd']
    
    if incon2 > 0:
        print(f"   ⚠️ Inconsistência 2: {incon2:,} empresas com periodos_omissos_normal>0 mas flag_omissao_dime_normal=0")
    else:
        print(f"   ✅ OK: Períodos omissos Normal consistentes com flag")
except:
    print(f"   ⚠️ Não foi possível verificar inconsistência 2")

# Inconsistência 3: Flags com nomenclatura errada
try:
    incon3 = spark.sql("""
        SELECT 
            SUM(CASE WHEN flag_omissao_pgdas_normal = 1 THEN 1 ELSE 0 END) as pgdas_normal,
            SUM(CASE WHEN flag_omissao_dime_simples = 1 THEN 1 ELSE 0 END) as dime_simples
        FROM teste.credito_dime_completo
    """).collect()[0]
    
    if incon3['pgdas_normal'] > 0:
        print(f"   ⚠️ Inconsistência 3: {incon3['pgdas_normal']:,} empresas com flag_omissao_pgdas_normal=1 (INCORRETO)")
    else:
        print(f"   ✅ OK: Nenhuma flag_omissao_pgdas_normal=1 (correto, PGDAS só tem Simples)")
        
    if incon3['dime_simples'] > 0:
        print(f"   ⚠️ Inconsistência 4: {incon3['dime_simples']:,} empresas com flag_omissao_dime_simples=1 (INCORRETO)")
    else:
        print(f"   ✅ OK: Nenhuma flag_omissao_dime_simples=1 (correto, DIME só tem Normal)")
except:
    print(f"   ⚠️ Não foi possível verificar inconsistências 3 e 4")

# ============================================================================
# 7. DISTRIBUIÇÃO POR FAIXAS
# ============================================================================
print("\n" + "="*60)
print("7. DISTRIBUIÇÃO POR FAIXAS DE PERÍODOS")
print("="*60)

try:
    dist = spark.sql("""
        SELECT 
            'Zerados Normal' as tipo,
            CASE 
                WHEN periodos_zerados_normal = 0 THEN '0'
                WHEN periodos_zerados_normal BETWEEN 1 AND 3 THEN '1-3'
                WHEN periodos_zerados_normal BETWEEN 4 AND 6 THEN '4-6'
                WHEN periodos_zerados_normal BETWEEN 7 AND 11 THEN '7-11'
                ELSE '12+'
            END as faixa,
            COUNT(*) as qtd
        FROM teste.credito_dime_completo
        GROUP BY tipo, faixa
        
        UNION ALL
        
        SELECT 
            'Omissos Normal' as tipo,
            CASE 
                WHEN periodos_omissos_normal = 0 THEN '0'
                WHEN periodos_omissos_normal BETWEEN 1 AND 3 THEN '1-3'
                WHEN periodos_omissos_normal BETWEEN 4 AND 6 THEN '4-6'
                WHEN periodos_omissos_normal BETWEEN 7 AND 11 THEN '7-11'
                ELSE '12+'
            END as faixa,
            COUNT(*) as qtd
        FROM teste.credito_dime_completo
        GROUP BY tipo, faixa
        
        UNION ALL
        
        SELECT 
            'Zerados Simples' as tipo,
            CASE 
                WHEN periodos_zerados_simples = 0 THEN '0'
                WHEN periodos_zerados_simples BETWEEN 1 AND 3 THEN '1-3'
                WHEN periodos_zerados_simples BETWEEN 4 AND 6 THEN '4-6'
                WHEN periodos_zerados_simples BETWEEN 7 AND 11 THEN '7-11'
                ELSE '12+'
            END as faixa,
            COUNT(*) as qtd
        FROM teste.credito_dime_completo
        GROUP BY tipo, faixa
        
        UNION ALL
        
        SELECT 
            'Omissos Simples' as tipo,
            CASE 
                WHEN periodos_omissos_simples = 0 THEN '0'
                WHEN periodos_omissos_simples BETWEEN 1 AND 3 THEN '1-3'
                WHEN periodos_omissos_simples BETWEEN 4 AND 6 THEN '4-6'
                WHEN periodos_omissos_simples BETWEEN 7 AND 11 THEN '7-11'
                ELSE '12+'
            END as faixa,
            COUNT(*) as qtd
        FROM teste.credito_dime_completo
        GROUP BY tipo, faixa
        ORDER BY tipo, faixa
    """).toPandas()
    
    print("\n📊 Distribuição por faixas:")
    for tipo in dist['tipo'].unique():
        print(f"\n   {tipo}:")
        df_tipo = dist[dist['tipo'] == tipo]
        for _, row in df_tipo.iterrows():
            print(f"      {row['faixa']:>5}: {int(row['qtd']):>10,} empresas")
            
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 8. AMOSTRA DE DADOS
# ============================================================================
print("\n" + "="*60)
print("8. AMOSTRA DE EMPRESAS")
print("="*60)

print("\n🔍 Top 10 empresas com mais declarações problemáticas:")

try:
    amostra = spark.sql("""
        SELECT 
            nu_cnpj,
            nm_razao_social,
            periodos_zerados_normal,
            periodos_zerados_simples,
            periodos_omissos_normal,
            periodos_omissos_simples,
            flag_omissao_dime_normal,
            flag_omissao_dime_simples,
            flag_omissao_pgdas_normal,
            flag_omissao_pgdas_simples,
            (periodos_zerados_normal + periodos_zerados_simples + 
             periodos_omissos_normal + periodos_omissos_simples) as total
        FROM teste.credito_dime_completo
        WHERE (periodos_zerados_normal + periodos_zerados_simples + 
               periodos_omissos_normal + periodos_omissos_simples) > 0
        ORDER BY total DESC
        LIMIT 10
    """).toPandas()
    
    print("\n   " + "-" * 130)
    print(f"   {'CNPJ':<15} {'Razão Social':<30} {'Z_Nor':>6} {'Z_Sim':>6} {'O_Nor':>6} {'O_Sim':>6} {'Flags (DN|DS|PN|PS)':>20}")
    print("   " + "-" * 130)
    for _, row in amostra.iterrows():
        flags = f"{int(row['flag_omissao_dime_normal'] or 0)}{int(row['flag_omissao_dime_simples'] or 0)}{int(row['flag_omissao_pgdas_normal'] or 0)}{int(row['flag_omissao_pgdas_simples'] or 0)}"
        razao = (row['nm_razao_social'][:27] + '...') if len(str(row['nm_razao_social'])) > 30 else str(row['nm_razao_social'])
        print(f"   {row['nu_cnpj']:<15} {razao:<30} {int(row['periodos_zerados_normal']):>6} {int(row['periodos_zerados_simples']):>6} {int(row['periodos_omissos_normal']):>6} {int(row['periodos_omissos_simples']):>6} {flags:>20}")
    print("   " + "-" * 130)
    print("   Legenda: Z_Nor=Zerados Normal | Z_Sim=Zerados Simples | O_Nor=Omissos Normal | O_Sim=Omissos Simples")
    print("   Flags: DN=DIME Normal | DS=DIME Simples | PN=PGDAS Normal | PS=PGDAS Simples")
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 9. RESUMO E RECOMENDAÇÕES
# ============================================================================
print("\n" + "="*60)
print("9. RESUMO E RECOMENDAÇÕES")
print("="*60)

print("""
📋 ENTENDIMENTO CONCEITUAL CORRETO:
   • DIME = Declaração do Regime NORMAL (não-Simples)
   • PGDAS = Declaração do Regime SIMPLES NACIONAL
   
🎯 ANÁLISE DE NOMENCLATURA DAS FLAGS:
   
   Situação ATUAL na tabela credito_dime_completo:
   ✅ flag_omissao_dime_normal   → CORRETO (DIME é do Normal)
   ❌ flag_omissao_dime_simples  → INCORRETO (DIME não existe no Simples)
   ❌ flag_omissao_pgdas_normal  → INCORRETO (PGDAS não existe no Normal)
   ✅ flag_omissao_pgdas_simples → CORRETO (PGDAS é do Simples)
   
💡 NOMENCLATURA CORRETA DEVERIA SER:
   Opção 1 (Simplificada - RECOMENDADA):
      • flag_omissao_dime   (remove "_normal" pois é redundante)
      • flag_omissao_pgdas  (remove "_simples" pois é redundante)
   
   Opção 2 (Manter padrão atual, corrigindo os erros):
      • flag_omissao_dime_normal   (mantém)
      • flag_omissao_pgdas_simples (mantém)
      • REMOVER: flag_omissao_dime_simples
      • REMOVER: flag_omissao_pgdas_normal

📊 COLUNAS DE PERÍODOS (estas estão corretas):
   ✅ periodos_zerados_normal   → Períodos zerados do Regime Normal (DIME)
   ✅ periodos_omissos_normal   → Períodos omissos do Regime Normal (DIME)
   ✅ periodos_zerados_simples  → Períodos zerados do Simples Nacional (PGDAS)
   ✅ periodos_omissos_simples  → Períodos omissos do Simples Nacional (PGDAS)

🔧 AÇÕES RECOMENDADAS NO SQL:
   1. Remover as colunas flag_omissao_dime_simples e flag_omissao_pgdas_normal
   2. Renomear flag_omissao_dime_normal para flag_omissao_dime
   3. Renomear flag_omissao_pgdas_simples para flag_omissao_pgdas
   4. Verificar se as tabelas de origem também têm essa inconsistência
""")

print("\n" + "="*80)
print("FIM DO DIAGNÓSTICO")
print("="*80)

DIAGNÓSTICO: DECLARAÇÕES ZERADAS E OMISSÕES
DIME (Regime Normal) vs PGDAS (Simples Nacional)

1. ESTRUTURA DAS TABELAS

📋 Regime Normal (DIME) (teste.cancel_zero_normal):
   ✓ periodos_omissos
   ✓ periodos_zerados
   ✓ periodos_sem_movimento
   ✓ total_periodos
   ✓ periodos_declarados
   ✓ qtde_indicios_fraude
   ✓ soma_score_indicios
   ✓ flag_omissao_dime_indicio
   ✓ flag_omissao_pgdas_indicio


   📊 Total de registros: 451,401

📋 Simples Nacional (PGDAS) (teste.cancel_zero_simples):
   ✓ periodos_omissos
   ✓ periodos_zerados
   ✓ total_periodos
   ✓ periodos_declarados
   ✓ qtde_indicios_fraude
   ✓ soma_score_indicios
   ✓ flag_omissao_dime_indicio
   ✓ flag_omissao_pgdas_indicio
   📊 Total de registros: 613,509

📋 Tabela Principal Consolidada (teste.credito_dime_completo):
   ✓ valor_igual_total_periodos_60m
   ✓ primeiro_periodo_valor_igual_60m
   ✓ ultimo_periodo_valor_igual_60m
   ✓ valor_igual_total_periodos_12m
   ✓ primeiro_periodo_valor_igual_12m
   ✓ ultimo_periodo_valor_igual_12m
   ✓ periodo_inicial_sequencia_12m
   ✓ periodo_final_sequencia_12m
   ✓ periodos_omissos_normal
   ✓ periodos_zerados_normal
   ✓ total_periodos_normal
   ✓ periodos_declarados_normal
   ✓ qtde_indicios_normal
   ✓ soma_score_indicios_normal
   ✓ flag_omissao_dime_normal
   ✓ flag_omissao_pgdas_normal
   ✓ periodos_omissos_simples
   ✓ periodos_zerados_simples
   ✓ total_periodos_simples


📊 Resumo Geral:
   Total de registros: 451,401
   Total de CNPJs únicos: 451,401

📋 Períodos:
   Com períodos omissos: 418,654 (92.7%)
   Com períodos zerados: 50,247 (11.1%)
   Com sem movimento: 0 (0.0%)
   Média omissos: 47.54
   Média zerados: 1.07
   Máximo omissos: 57
   Máximo zerados: 57

🔍 Indícios:
   Com indícios de fraude: 91,232
   Flag omissão DIME: 12,497
   Flag omissão PGDAS: 675 ⚠️ (DEVERIA SER 0 - NORMAL NÃO TEM PGDAS)
   Média de indícios: 1.85
   Média score: 189.45

3. ANÁLISE: cancel_zero_simples (SIMPLES NACIONAL - PGDAS)



📊 Resumo Geral:
   Total de registros: 613,509
   Total de CNPJs únicos: 613,509

📋 Períodos:
   Com períodos omissos: 510,468 (83.2%)
   Com períodos zerados: 145,416 (23.7%)
   Média omissos: 41.68
   Média zerados: 3.01
   Máximo omissos: 57
   Máximo zerados: 57

🔍 Indícios:
   Com indícios de fraude: 323,074
   Flag omissão DIME: 1 ⚠️ (DEVERIA SER 0 - SIMPLES NÃO TEM DIME)
   Flag omissão PGDAS: 4,723
   Média de indícios: 3.32
   Média score: 356.42

4. ANÁLISE: credito_dime_completo (TABELA PRINCIPAL)

📊 Total de empresas: 54,777

🔴 FLAGS DE OMISSÃO:
   flag_omissao_dime_normal: 3,430 ✅ (CORRETO)
   flag_omissao_dime_simples: 0 ⚠️ (INCORRETO - DIME NÃO TEM SIMPLES)
   flag_omissao_pgdas_normal: 22 ⚠️ (INCORRETO - PGDAS NÃO TEM NORMAL)
   flag_omissao_pgdas_simples: 82 ✅ (CORRETO)

📋 PERÍODOS - REGIME NORMAL:
   Com zerados: 27,719 (média: 4.95)
   Com omissos: 31,754 (média: 17.80)

📋 PERÍODOS - SIMPLES NACIONAL:
   Com zerados: 2,406 (média: 0.55)
   Com omissos: 3,555 (média:


📊 CNPJs nas tabelas de origem:
   cancel_zero_normal (com declarações): 429,920
   cancel_zero_simples (com declarações): 547,284

📊 CNPJs na tabela destino:
   credito_dime_completo (com declarações): 42,867

✅ Matches Normal:
   38,906 / 429,920 (9.0%)

✅ Matches Simples:
   3,961 / 547,284 (0.7%)

6. IDENTIFICAR INCONSISTÊNCIAS

🔍 Verificando casos problemáticos:
   ⚠️ Inconsistência 1: 862 empresas com flag_omissao_dime_normal=1 mas periodos_omissos_normal=0
   ⚠️ Inconsistência 2: 29,186 empresas com periodos_omissos_normal>0 mas flag_omissao_dime_normal=0
   ⚠️ Inconsistência 3: 22 empresas com flag_omissao_pgdas_normal=1 (INCORRETO)
   ✅ OK: Nenhuma flag_omissao_dime_simples=1 (correto, DIME só tem Normal)

7. DISTRIBUIÇÃO POR FAIXAS DE PERÍODOS

📊 Distribuição por faixas:

   Omissos Normal:
          0:     23,023 empresas
        1-3:        945 empresas
        12+:     27,971 empresas
        4-6:        907 empresas
       7-11:      1,931 empresas

   Omissos Simples:
  

In [8]:
"""
DIAGNÓSTICO DAS TABELAS FONTES ORIGINAIS
Validação de dados de entrada para cancel_zero_normal e cancel_zero_simples
"""

print("="*80)
print("DIAGNÓSTICO: TABELAS FONTES ORIGINAIS")
print("="*80)

# ============================================================================
# 1. ANÁLISE DA FONTE: ods_decl_dime_raw (DECLARAÇÕES DIME - REGIME NORMAL)
# ============================================================================
print("\n" + "="*60)
print("1. FONTE: usr_sat_ods.ods_decl_dime_raw (DECLARAÇÕES DIME)")
print("="*60)

try:
    # Verificar estrutura
    print("\n📋 Estrutura da tabela:")
    colunas_dime = spark.sql("DESCRIBE usr_sat_ods.ods_decl_dime_raw").collect()
    colunas_importantes = [col.col_name for col in colunas_dime if any(x in col.col_name.upper() 
                          for x in ['CNPJ', 'PER_REF', 'FATURAMENTO', 'RECEITA', 'CRED', 'DEB'])]
    for col in colunas_importantes:
        print(f"   ✓ {col}")
    
    # Estatísticas gerais
    print("\n📊 Estatísticas Gerais (período 202101-202509):")
    stats_dime = spark.sql("""
        SELECT 
            COUNT(*) as total_registros,
            COUNT(DISTINCT nu_cnpj) as total_cnpjs,
            MIN(nu_per_ref) as primeiro_periodo,
            MAX(nu_per_ref) as ultimo_periodo,
            COUNT(DISTINCT nu_per_ref) as qtde_periodos_distintos
        FROM usr_sat_ods.ods_decl_dime_raw
        WHERE nu_per_ref BETWEEN 202101 AND 202509
    """).collect()[0]
    
    print(f"   Total de registros: {stats_dime['total_registros']:,}")
    print(f"   Total de CNPJs únicos: {stats_dime['total_cnpjs']:,}")
    print(f"   Primeiro período: {stats_dime['primeiro_periodo']}")
    print(f"   Último período: {stats_dime['ultimo_periodo']}")
    print(f"   Períodos distintos: {stats_dime['qtde_periodos_distintos']}")
    
    # Análise de declarações zeradas
    print("\n🔍 Análise de Declarações Zeradas:")
    zeradas_dime = spark.sql("""
        SELECT 
            -- Critério ATUAL do SQL
            SUM(CASE WHEN VL_FATURAMENTO = 0 THEN 1 ELSE 0 END) as apenas_faturamento_zero,
            SUM(CASE WHEN VL_TOT_CRED = 0 AND VL_TOT_DEB = 0 THEN 1 ELSE 0 END) as cred_deb_zero,
            SUM(CASE WHEN VL_FATURAMENTO = 0 OR (VL_TOT_CRED = 0 AND VL_TOT_DEB = 0) THEN 1 ELSE 0 END) as criterio_atual,
            
            -- Critério ALTERNATIVO (mais restritivo)
            SUM(CASE WHEN VL_RECEITA_BRUTA = 0 AND VL_FATURAMENTO = 0 
                          AND VL_TOT_CRED = 0 AND VL_TOT_DEB = 0 
                          AND VL_DEB_RECOLHER = 0 THEN 1 ELSE 0 END) as sem_movimento_total,
            
            -- Análise de campos NULL
            SUM(CASE WHEN VL_FATURAMENTO IS NULL THEN 1 ELSE 0 END) as faturamento_null,
            SUM(CASE WHEN VL_TOT_CRED IS NULL THEN 1 ELSE 0 END) as cred_null,
            SUM(CASE WHEN VL_TOT_DEB IS NULL THEN 1 ELSE 0 END) as deb_null,
            
            COUNT(*) as total
        FROM usr_sat_ods.ods_decl_dime_raw
        WHERE nu_per_ref BETWEEN 202101 AND 202509
    """).collect()[0]
    
    print(f"   Apenas faturamento = 0: {zeradas_dime['apenas_faturamento_zero']:,} ({zeradas_dime['apenas_faturamento_zero']/zeradas_dime['total']*100:.1f}%)")
    print(f"   Crédito E Débito = 0: {zeradas_dime['cred_deb_zero']:,} ({zeradas_dime['cred_deb_zero']/zeradas_dime['total']*100:.1f}%)")
    print(f"   Critério ATUAL (Fat=0 OU Cred=Deb=0): {zeradas_dime['criterio_atual']:,} ({zeradas_dime['criterio_atual']/zeradas_dime['total']*100:.1f}%)")
    print(f"   Sem movimento total: {zeradas_dime['sem_movimento_total']:,} ({zeradas_dime['sem_movimento_total']/zeradas_dime['total']*100:.1f}%)")
    print(f"   Faturamento NULL: {zeradas_dime['faturamento_null']:,}")
    print(f"   Crédito NULL: {zeradas_dime['cred_null']:,}")
    print(f"   Débito NULL: {zeradas_dime['deb_null']:,}")
    
    # Distribuição por período
    print("\n📅 Registros por período (últimos 12 meses):")
    dist_periodo = spark.sql("""
        SELECT 
            nu_per_ref as periodo,
            COUNT(*) as qtd_registros,
            COUNT(DISTINCT nu_cnpj) as qtd_cnpjs,
            SUM(CASE WHEN VL_FATURAMENTO = 0 OR (VL_TOT_CRED = 0 AND VL_TOT_DEB = 0) THEN 1 ELSE 0 END) as qtd_zerados
        FROM usr_sat_ods.ods_decl_dime_raw
        WHERE nu_per_ref BETWEEN 202409 AND 202509
        GROUP BY nu_per_ref
        ORDER BY nu_per_ref DESC
    """).toPandas()
    
    for _, row in dist_periodo.iterrows():
        print(f"   {row['periodo']}: {int(row['qtd_registros']):>7,} registros | {int(row['qtd_cnpjs']):>7,} CNPJs | {int(row['qtd_zerados']):>6,} zerados")
    
    # Amostra de declarações zeradas
    print("\n🔬 Amostra de 5 declarações consideradas ZERADAS:")
    amostra_zeradas = spark.sql("""
        SELECT 
            nu_cnpj,
            nu_per_ref,
            VL_RECEITA_BRUTA,
            VL_FATURAMENTO,
            VL_TOT_CRED,
            VL_TOT_DEB,
            VL_DEB_RECOLHER
        FROM usr_sat_ods.ods_decl_dime_raw
        WHERE nu_per_ref BETWEEN 202101 AND 202509
          AND (VL_FATURAMENTO = 0 OR (VL_TOT_CRED = 0 AND VL_TOT_DEB = 0))
        LIMIT 5
    """).toPandas()
    
    print("\n   " + "-" * 110)
    print(f"   {'CNPJ':<15} {'Período':<10} {'Rec.Bruta':>12} {'Faturamento':>12} {'Crédito':>12} {'Débito':>12} {'Deb.Rec':>12}")
    print("   " + "-" * 110)
    for _, row in amostra_zeradas.iterrows():
        print(f"   {row['nu_cnpj']:<15} {row['nu_per_ref']:<10} {row['VL_RECEITA_BRUTA']:>12,.2f} {row['VL_FATURAMENTO']:>12,.2f} {row['VL_TOT_CRED']:>12,.2f} {row['VL_TOT_DEB']:>12,.2f} {row['VL_DEB_RECOLHER']:>12,.2f}")
    print("   " + "-" * 110)
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 2. ANÁLISE DA FONTE: vw_sna_pgdasd_grupo_empresarial (PGDAS - SIMPLES)
# ============================================================================
print("\n" + "="*60)
print("2. FONTE: usr_sat_ods.vw_sna_pgdasd_grupo_empresarial (PGDAS)")
print("="*60)

try:
    # Verificar estrutura
    print("\n📋 Estrutura da tabela:")
    colunas_pgdas = spark.sql("DESCRIBE usr_sat_ods.vw_sna_pgdasd_grupo_empresarial").collect()
    colunas_importantes = [col.col_name for col in colunas_pgdas if any(x in col.col_name.lower() 
                          for x in ['cnpj', 'per_ref', 'rec_bruta', 'icms'])]
    for col in colunas_importantes:
        print(f"   ✓ {col}")
    
    # Estatísticas gerais
    print("\n📊 Estatísticas Gerais (período 202101-202509):")
    stats_pgdas = spark.sql("""
        SELECT 
            COUNT(*) as total_registros,
            COUNT(DISTINCT nu_cnpj_matriz) as total_cnpjs,
            MIN(nu_per_ref) as primeiro_periodo,
            MAX(nu_per_ref) as ultimo_periodo,
            COUNT(DISTINCT nu_per_ref) as qtde_periodos_distintos
        FROM usr_sat_ods.vw_sna_pgdasd_grupo_empresarial
        WHERE nu_per_ref BETWEEN 202101 AND 202509
    """).collect()[0]
    
    print(f"   Total de registros: {stats_pgdas['total_registros']:,}")
    print(f"   Total de CNPJs únicos: {stats_pgdas['total_cnpjs']:,}")
    print(f"   Primeiro período: {stats_pgdas['primeiro_periodo']}")
    print(f"   Último período: {stats_pgdas['ultimo_periodo']}")
    print(f"   Períodos distintos: {stats_pgdas['qtde_periodos_distintos']}")
    
    # Análise de declarações zeradas
    print("\n🔍 Análise de Declarações Zeradas:")
    zeradas_pgdas = spark.sql("""
        SELECT 
            SUM(CASE WHEN vl_rec_bruta = 0 THEN 1 ELSE 0 END) as rec_bruta_zero,
            SUM(CASE WHEN vl_rec_bruta IS NULL THEN 1 ELSE 0 END) as rec_bruta_null,
            SUM(CASE WHEN vl_icms_sc = 0 THEN 1 ELSE 0 END) as icms_zero,
            SUM(CASE WHEN vl_icms_sc IS NULL THEN 1 ELSE 0 END) as icms_null,
            COUNT(*) as total
        FROM usr_sat_ods.vw_sna_pgdasd_grupo_empresarial
        WHERE nu_per_ref BETWEEN 202101 AND 202509
    """).collect()[0]
    
    print(f"   Receita Bruta = 0: {zeradas_pgdas['rec_bruta_zero']:,} ({zeradas_pgdas['rec_bruta_zero']/zeradas_pgdas['total']*100:.1f}%)")
    print(f"   Receita Bruta NULL: {zeradas_pgdas['rec_bruta_null']:,}")
    print(f"   ICMS SC = 0: {zeradas_pgdas['icms_zero']:,} ({zeradas_pgdas['icms_zero']/zeradas_pgdas['total']*100:.1f}%)")
    print(f"   ICMS SC NULL: {zeradas_pgdas['icms_null']:,}")
    
    # Distribuição por período
    print("\n📅 Registros por período (últimos 12 meses):")
    dist_periodo_pgdas = spark.sql("""
        SELECT 
            nu_per_ref as periodo,
            COUNT(*) as qtd_registros,
            COUNT(DISTINCT nu_cnpj_matriz) as qtd_cnpjs,
            SUM(CASE WHEN vl_rec_bruta = 0 THEN 1 ELSE 0 END) as qtd_zerados
        FROM usr_sat_ods.vw_sna_pgdasd_grupo_empresarial
        WHERE nu_per_ref BETWEEN 202409 AND 202509
        GROUP BY nu_per_ref
        ORDER BY nu_per_ref DESC
    """).toPandas()
    
    for _, row in dist_periodo_pgdas.iterrows():
        print(f"   {row['periodo']}: {int(row['qtd_registros']):>7,} registros | {int(row['qtd_cnpjs']):>7,} CNPJs | {int(row['qtd_zerados']):>6,} zerados")
    
    # Amostra de declarações zeradas
    print("\n🔬 Amostra de 5 declarações consideradas ZERADAS:")
    amostra_zeradas_pgdas = spark.sql("""
        SELECT 
            nu_cnpj_matriz,
            nu_per_ref,
            vl_rec_bruta,
            vl_rec_bruta_em_12m,
            vl_rec_bruta_no_ano,
            vl_icms_sc
        FROM usr_sat_ods.vw_sna_pgdasd_grupo_empresarial
        WHERE nu_per_ref BETWEEN 202101 AND 202509
          AND vl_rec_bruta = 0
        LIMIT 5
    """).toPandas()
    
    print("\n   " + "-" * 100)
    print(f"   {'CNPJ':<15} {'Período':<10} {'Rec.Bruta':>12} {'Rec.12m':>12} {'Rec.Ano':>12} {'ICMS SC':>12}")
    print("   " + "-" * 100)
    for _, row in amostra_zeradas_pgdas.iterrows():
        print(f"   {row['nu_cnpj_matriz']:<15} {row['nu_per_ref']:<10} {row['vl_rec_bruta']:>12,.2f} {row['vl_rec_bruta_em_12m']:>12,.2f} {row['vl_rec_bruta_no_ano']:>12,.2f} {row['vl_icms_sc']:>12,.2f}")
    print("   " + "-" * 100)
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 3. ANÁLISE: vw_cad_contrib (CADASTRO DE CONTRIBUINTES)
# ============================================================================
print("\n" + "="*60)
print("3. FONTE: usr_sat_ods.vw_cad_contrib (CADASTRO)")
print("="*60)

try:
    # Estatísticas de regime
    print("\n📊 Distribuição por Regime de Apuração:")
    regimes = spark.sql("""
        SELECT 
            nm_reg_apuracao,
            COUNT(*) as qtd
        FROM usr_sat_ods.vw_cad_contrib
        GROUP BY nm_reg_apuracao
        ORDER BY qtd DESC
    """).toPandas()
    
    total_cadastro = regimes['qtd'].sum()
    for _, row in regimes.iterrows():
        regime = row['nm_reg_apuracao'] if row['nm_reg_apuracao'] else 'NULL'
        print(f"   {regime:<30}: {int(row['qtd']):>10,} ({row['qtd']/total_cadastro*100:.1f}%)")
    
    # Verificar se há CNPJs que mudaram de regime
    print("\n🔄 Verificar mudanças de regime (amostra):")
    print("   (Verificando se a lógica de filtrar por regime ATUAL está correta)")
    
    mudancas = spark.sql("""
        WITH regime_atual AS (
            SELECT 
                nu_cnpj,
                nm_reg_apuracao,
                dt_inicio_atividade
            FROM usr_sat_ods.vw_cad_contrib
            WHERE nm_reg_apuracao IS NOT NULL
        )
        SELECT 
            nm_reg_apuracao,
            COUNT(*) as qtd,
            MIN(dt_inicio_atividade) as dt_mais_antiga,
            MAX(dt_inicio_atividade) as dt_mais_recente
        FROM regime_atual
        GROUP BY nm_reg_apuracao
    """).toPandas()
    
    for _, row in mudancas.iterrows():
        regime = row['nm_reg_apuracao'] if row['nm_reg_apuracao'] else 'NULL'
        print(f"   {regime}: {int(row['qtd']):,} empresas")
        if row['dt_mais_antiga'] and row['dt_mais_recente']:
            print(f"      Período de início: {row['dt_mais_antiga']} a {row['dt_mais_recente']}")
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 4. ANÁLISE: neaf.empresa_indicio (INDÍCIOS DE FRAUDE)
# ============================================================================
print("\n" + "="*60)
print("4. FONTE: neaf.empresa_indicio (INDÍCIOS DE FRAUDE)")
print("="*60)

try:
    # Estatísticas gerais
    print("\n📊 Estatísticas Gerais:")
    stats_indicios = spark.sql("""
        SELECT 
            COUNT(*) as total_registros,
            COUNT(DISTINCT nu_cpf_cnpj) as total_cnpjs,
            COUNT(DISTINCT cd_indicio) as total_indicios_distintos,
            SUM(CASE WHEN cd_atual = 1 THEN 1 ELSE 0 END) as indicios_atuais,
            SUM(CASE WHEN cd_uf = 'SC' THEN 1 ELSE 0 END) as indicios_sc
        FROM neaf.empresa_indicio
    """).collect()[0]
    
    print(f"   Total de registros: {stats_indicios['total_registros']:,}")
    print(f"   Total de CNPJs únicos: {stats_indicios['total_cnpjs']:,}")
    print(f"   Indícios distintos: {stats_indicios['total_indicios_distintos']}")
    print(f"   Indícios atuais (cd_atual=1): {stats_indicios['indicios_atuais']:,}")
    print(f"   Indícios SC (cd_uf='SC'): {stats_indicios['indicios_sc']:,}")
    
    # Distribuição por tipo de indício
    print("\n📋 Distribuição por Código de Indício (SC, atuais):")
    dist_indicios = spark.sql("""
        SELECT 
            cd_indicio,
            COUNT(*) as qtd,
            COUNT(DISTINCT nu_cpf_cnpj) as qtd_cnpjs,
            ROUND(AVG(nu_score), 2) as score_medio
        FROM neaf.empresa_indicio
        WHERE cd_uf = 'SC' AND cd_atual = 1
        GROUP BY cd_indicio
        ORDER BY qtd DESC
        LIMIT 20
    """).toPandas()
    
    for _, row in dist_indicios.iterrows():
        print(f"   Indício {int(row['cd_indicio']):>3}: {int(row['qtd']):>8,} ocorrências | {int(row['qtd_cnpjs']):>8,} CNPJs | score médio: {row['score_medio']:.2f}")
    
    # Verificar os indícios 11 (DIME) e 12 (PGDAS)
    print("\n🔍 Análise dos Indícios 11 (DIME) e 12 (PGDAS):")
    ind_11_12 = spark.sql("""
        SELECT 
            cd_indicio,
            COUNT(*) as qtd,
            COUNT(DISTINCT nu_cpf_cnpj) as qtd_cnpjs,
            ROUND(AVG(nu_score), 2) as score_medio,
            MIN(nu_score) as score_min,
            MAX(nu_score) as score_max
        FROM neaf.empresa_indicio
        WHERE cd_uf = 'SC' 
          AND cd_atual = 1
          AND cd_indicio IN (11, 12)
        GROUP BY cd_indicio
    """).toPandas()
    
    if len(ind_11_12) > 0:
        for _, row in ind_11_12.iterrows():
            tipo = "DIME (omissão)" if row['cd_indicio'] == 11 else "PGDAS (omissão)"
            print(f"   Indício {int(row['cd_indicio'])} ({tipo}):")
            print(f"      Ocorrências: {int(row['qtd']):,}")
            print(f"      CNPJs únicos: {int(row['qtd_cnpjs']):,}")
            print(f"      Score médio: {row['score_medio']:.2f}")
            print(f"      Score range: {row['score_min']:.2f} - {row['score_max']:.2f}")
    else:
        print("   ⚠️ Nenhum registro encontrado para indícios 11 e 12")
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 5. VALIDAÇÃO: MATRIZ DE PERÍODOS
# ============================================================================
print("\n" + "="*60)
print("5. VALIDAÇÃO: MATRIZ DE PERÍODOS (202101-202509)")
print("="*60)

try:
    print("\n📅 Períodos esperados: 57 períodos (202101 a 202509)")
    print("   2021: 12 meses | 2022: 12 meses | 2023: 12 meses | 2024: 12 meses | 2025: 9 meses")
    
    # Verificar se todos os períodos existem nas fontes
    print("\n🔍 Verificando cobertura de períodos na DIME:")
    periodos_dime = spark.sql("""
        SELECT 
            nu_per_ref,
            COUNT(DISTINCT nu_cnpj) as qtd_cnpjs
        FROM usr_sat_ods.ods_decl_dime_raw
        WHERE nu_per_ref BETWEEN 202101 AND 202509
        GROUP BY nu_per_ref
        ORDER BY nu_per_ref
    """).toPandas()
    
    periodos_esperados = list(range(202101, 202113)) + list(range(202201, 202213)) + \
                        list(range(202301, 202313)) + list(range(202401, 202413)) + \
                        list(range(202501, 202510))
    
    periodos_encontrados = periodos_dime['nu_per_ref'].tolist()
    periodos_faltantes = set(periodos_esperados) - set(periodos_encontrados)
    
    if periodos_faltantes:
        print(f"   ⚠️ Períodos FALTANTES na DIME: {sorted(periodos_faltantes)}")
    else:
        print(f"   ✅ Todos os 57 períodos presentes na DIME")
    
    print(f"   📊 Total de períodos encontrados: {len(periodos_encontrados)}")
    
    # Mesma verificação para PGDAS
    print("\n🔍 Verificando cobertura de períodos no PGDAS:")
    periodos_pgdas = spark.sql("""
        SELECT 
            nu_per_ref,
            COUNT(DISTINCT nu_cnpj_matriz) as qtd_cnpjs
        FROM usr_sat_ods.vw_sna_pgdasd_grupo_empresarial
        WHERE nu_per_ref BETWEEN 202101 AND 202509
        GROUP BY nu_per_ref
        ORDER BY nu_per_ref
    """).toPandas()
    
    periodos_encontrados_pgdas = periodos_pgdas['nu_per_ref'].tolist()
    periodos_faltantes_pgdas = set(periodos_esperados) - set(periodos_encontrados_pgdas)
    
    if periodos_faltantes_pgdas:
        print(f"   ⚠️ Períodos FALTANTES no PGDAS: {sorted(periodos_faltantes_pgdas)}")
    else:
        print(f"   ✅ Todos os 57 períodos presentes no PGDAS")
    
    print(f"   📊 Total de períodos encontrados: {len(periodos_encontrados_pgdas)}")
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 6. ANÁLISE DE OMISSÕES: LÓGICA DE CÁLCULO
# ============================================================================
print("\n" + "="*60)
print("6. ANÁLISE DE OMISSÕES: VALIDAÇÃO DA LÓGICA")
print("="*60)

try:
    print("\n🔍 Testando a lógica de cálculo de omissões:")
    print("   (Pegando 3 CNPJs aleatórios e calculando suas omissões)")
    
    # Pegar 3 CNPJs aleatórios que têm declarações
    cnpjs_teste = spark.sql("""
        SELECT DISTINCT nu_cnpj
        FROM usr_sat_ods.ods_decl_dime_raw
        WHERE nu_per_ref BETWEEN 202101 AND 202509
        ORDER BY RAND()
        LIMIT 3
    """).toPandas()['nu_cnpj'].tolist()
    
    for cnpj in cnpjs_teste:
        print(f"\n   📊 CNPJ: {cnpj}")
        
        # Calcular períodos declarados e omissos
        analise = spark.sql(f"""
            WITH periodos_esperados AS (
                SELECT periodo FROM (
                    SELECT 202101 AS periodo UNION ALL SELECT 202102 UNION ALL SELECT 202103 UNION ALL
                    SELECT 202104 UNION ALL SELECT 202105 UNION ALL SELECT 202106 UNION ALL SELECT 202107 UNION ALL
                    SELECT 202108 UNION ALL SELECT 202109 UNION ALL SELECT 202110 UNION ALL SELECT 202111 UNION ALL
                    SELECT 202112 UNION ALL SELECT 202201 UNION ALL SELECT 202202 UNION ALL SELECT 202203 UNION ALL
                    SELECT 202204 UNION ALL SELECT 202205 UNION ALL SELECT 202206 UNION ALL SELECT 202207 UNION ALL
                    SELECT 202208 UNION ALL SELECT 202209 UNION ALL SELECT 202210 UNION ALL SELECT 202211 UNION ALL
                    SELECT 202212 UNION ALL SELECT 202301 UNION ALL SELECT 202302 UNION ALL SELECT 202303 UNION ALL
                    SELECT 202304 UNION ALL SELECT 202305 UNION ALL SELECT 202306 UNION ALL SELECT 202307 UNION ALL
                    SELECT 202308 UNION ALL SELECT 202309 UNION ALL SELECT 202310 UNION ALL SELECT 202311 UNION ALL
                    SELECT 202312 UNION ALL SELECT 202401 UNION ALL SELECT 202402 UNION ALL SELECT 202403 UNION ALL
                    SELECT 202404 UNION ALL SELECT 202405 UNION ALL SELECT 202406 UNION ALL SELECT 202407 UNION ALL
                    SELECT 202408 UNION ALL SELECT 202409 UNION ALL SELECT 202410 UNION ALL SELECT 202411 UNION ALL
                    SELECT 202412 UNION ALL SELECT 202501 UNION ALL SELECT 202502 UNION ALL SELECT 202503 UNION ALL
                    SELECT 202504 UNION ALL SELECT 202505 UNION ALL SELECT 202506 UNION ALL SELECT 202507 UNION ALL
                    SELECT 202508 UNION ALL SELECT 202509
                ) p
            ),
            periodos_declarados AS (
                SELECT DISTINCT nu_per_ref AS periodo
                FROM usr_sat_ods.ods_decl_dime_raw
                WHERE nu_cnpj = '{cnpj}'
                  AND nu_per_ref BETWEEN 202101 AND 202509
            )
            SELECT 
                COUNT(DISTINCT pe.periodo) as total_periodos,
                COUNT(DISTINCT pd.periodo) as periodos_declarados,
                COUNT(DISTINCT pe.periodo) - COUNT(DISTINCT pd.periodo) as periodos_omissos
            FROM periodos_esperados pe
            LEFT JOIN periodos_declarados pd ON pe.periodo = pd.periodo
        """).collect()[0]
        
        print(f"      Total de períodos: {analise['total_periodos']}")
        print(f"      Períodos declarados: {analise['periodos_declarados']}")
        print(f"      Períodos omissos: {analise['periodos_omissos']}")
        
        # Mostrar alguns períodos declarados
        periodos_dec = spark.sql(f"""
            SELECT nu_per_ref, VL_FATURAMENTO, VL_TOT_CRED, VL_TOT_DEB
            FROM usr_sat_ods.ods_decl_dime_raw
            WHERE nu_cnpj = '{cnpj}'
              AND nu_per_ref BETWEEN 202101 AND 202509
            ORDER BY nu_per_ref DESC
            LIMIT 5
        """).toPandas()
        
        print(f"      Últimas 5 declarações:")
        for _, row in periodos_dec.iterrows():
            print(f"         {row['nu_per_ref']}: Fat={row['VL_FATURAMENTO']:.2f}, Cred={row['VL_TOT_CRED']:.2f}, Deb={row['VL_TOT_DEB']:.2f}")
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 7. RESUMO E CONCLUSÕES
# ============================================================================
print("\n" + "="*60)
print("7. RESUMO E CONCLUSÕES")
print("="*60)

print("""
🎯 PRINCIPAIS ACHADOS:

1️⃣ CRITÉRIO DE DECLARAÇÃO ZERADA (DIME):
   • Critério atual: VL_FATURAMENTO = 0 OU (VL_TOT_CRED = 0 AND VL_TOT_DEB = 0)
   • Este critério está CORRETO e captura declarações com "sem movimento"
   
2️⃣ CRITÉRIO DE DECLARAÇÃO ZERADA (PGDAS):
   • Critério atual: vl_rec_bruta = 0
   • Critério simples e direto, está CORRETO
   
3️⃣ MATRIZ DE PERÍODOS:
   • Verificar se todos os 57 períodos (202101-202509) estão presentes
   • Validar se há "buracos" na série temporal
   
4️⃣ INDÍCIOS DE FRAUDE:
   • Indício 11 = omissão DIME (Regime Normal)
   • Indício 12 = omissão PGDAS (Simples Nacional)
   • Verificar consistência entre as flags e os indícios
   
5️⃣ REGIME DE APURAÇÃO:
   • Filtro por regime ATUAL está correto
   • Empresas podem ter mudado de regime ao longo do tempo
   
⚠️ POSSÍVEIS PROBLEMAS A INVESTIGAR:

1. Se há períodos faltantes nas fontes originais
2. Se a lógica de "omissão" está considerando períodos corretos (57 vs 8?)
3. Se há CNPJs duplicados ou com formatos diferentes
4. Se os filtros de cd_uf='SC' e cd_atual=1 estão corretos para indícios

💡 PRÓXIMOS PASSOS:
   • Com base nos resultados acima, ajustar os SQLs se necessário
   • Validar especialmente a contagem de períodos (57 períodos esperados)
   • Verificar se "total_periodos" nas tabelas auxiliares está correto
""")

print("\n" + "="*80)
print("FIM DO DIAGNÓSTICO DAS FONTES ORIGINAIS")
print("="*80)

DIAGNÓSTICO: TABELAS FONTES ORIGINAIS

1. FONTE: usr_sat_ods.ods_decl_dime_raw (DECLARAÇÕES DIME)

📋 Estrutura da tabela:
   ✓ nu_cnpj
   ✓ nu_cnpj_grupo
   ✓ nu_per_ref
   ✓ vl_faturamento
   ✓ vl_cred_mes_anterior
   ✓ vl_cred_auc
   ✓ vl_cred_dcip
   ✓ vl_tot_cred
   ✓ vl_tot_deb
   ✓ vl_cred_ap_cons
   ✓ vl_deb_ap_cons
   ✓ vl_cred_mes_seguinte
   ✓ vl_deb_recolher
   ✓ vl_sdo_cred_exportacao
   ✓ vl_sdo_cred_isenta
   ✓ vl_sdo_cred_diferida
   ✓ vl_sdo_cred_outros
   ✓ vl_receita_bruta
   ✓ vl_tot_ent_imposto_creditado
   ✓ vl_tot_sai_imposto_creditado
   ✓ vl_faturamento_comunicacao

📊 Estatísticas Gerais (período 202101-202509):


   Total de registros: 4,428,586
   Total de CNPJs únicos: 165,616
   Primeiro período: 202101
   Último período: 202509
   Períodos distintos: 57

🔍 Análise de Declarações Zeradas:


   Apenas faturamento = 0: 457,812 (10.3%)
   Crédito E Débito = 0: 0 (0.0%)
   Critério ATUAL (Fat=0 OU Cred=Deb=0): 457,812 (10.3%)
   Sem movimento total: 0 (0.0%)
   Faturamento NULL: 1,633,194
   Crédito NULL: 1,994,781
   Débito NULL: 2,185,371

📅 Registros por período (últimos 12 meses):


   202509:  92,742 registros |  92,742 CNPJs |  9,876 zerados
   202508:  92,888 registros |  92,888 CNPJs | 10,077 zerados
   202507:  94,497 registros |  94,497 CNPJs | 10,300 zerados
   202506:  94,589 registros |  94,589 CNPJs | 10,284 zerados
   202505:  96,137 registros |  96,137 CNPJs | 10,542 zerados
   202504:  96,628 registros |  96,628 CNPJs | 10,423 zerados
   202503:  97,353 registros |  97,353 CNPJs | 10,475 zerados
   202502:  99,718 registros |  99,718 CNPJs | 10,842 zerados
   202501: 100,514 registros | 100,514 CNPJs | 10,969 zerados
   202412:  83,907 registros |  83,907 CNPJs |  9,773 zerados
   202411:  83,685 registros |  83,685 CNPJs |  9,788 zerados
   202410:  87,043 registros |  87,043 CNPJs |  9,869 zerados
   202409:  86,850 registros |  86,850 CNPJs |  9,862 zerados

🔬 Amostra de 5 declarações consideradas ZERADAS:

   --------------------------------------------------------------------------------------------------------------
   CNPJ            Período   

   Total de registros: 17,874,580
   Total de CNPJs únicos: 561,806
   Primeiro período: 202101
   Último período: 202509
   Períodos distintos: 57

🔍 Análise de Declarações Zeradas:


...

   ✅ Todos os 57 períodos presentes na DIME
   📊 Total de períodos encontrados: 57

🔍 Verificando cobertura de períodos no PGDAS:


   ✅ Todos os 57 períodos presentes no PGDAS
   📊 Total de períodos encontrados: 57

6. ANÁLISE DE OMISSÕES: VALIDAÇÃO DA LÓGICA

🔍 Testando a lógica de cálculo de omissões:
   (Pegando 3 CNPJs aleatórios e calculando suas omissões)



   📊 CNPJ: 37127086000128
      Total de períodos: 57
      Períodos declarados: 1
      Períodos omissos: 56


[Stage 192:==============================================>        (17 + 3) / 20]

      Últimas 5 declarações:
   ❌ Erro: unsupported format string passed to NoneType.__format__

7. RESUMO E CONCLUSÕES

🎯 PRINCIPAIS ACHADOS:

1️⃣ CRITÉRIO DE DECLARAÇÃO ZERADA (DIME):
   • Critério atual: VL_FATURAMENTO = 0 OU (VL_TOT_CRED = 0 AND VL_TOT_DEB = 0)
   • Este critério está CORRETO e captura declarações com "sem movimento"
   
2️⃣ CRITÉRIO DE DECLARAÇÃO ZERADA (PGDAS):
   • Critério atual: vl_rec_bruta = 0
   • Critério simples e direto, está CORRETO
   
3️⃣ MATRIZ DE PERÍODOS:
   • Verificar se todos os 57 períodos (202101-202509) estão presentes
   • Validar se há "buracos" na série temporal
   
4️⃣ INDÍCIOS DE FRAUDE:
   • Indício 11 = omissão DIME (Regime Normal)
   • Indício 12 = omissão PGDAS (Simples Nacional)
   • Verificar consistência entre as flags e os indícios
   
5️⃣ REGIME DE APURAÇÃO:
   • Filtro por regime ATUAL está correto
   • Empresas podem ter mudado de regime ao longo do tempo
   
⚠️ POSSÍVEIS PROBLEMAS A INVESTIGAR:

1. Se há períodos faltantes na

In [9]:
"""
INVESTIGAÇÃO APROFUNDADA: PROBLEMAS IDENTIFICADOS
Foco em: NULLs, Inconsistência de Flags e Joins
"""

print("="*80)
print("INVESTIGAÇÃO: PROBLEMAS CRÍTICOS IDENTIFICADOS")
print("="*80)

# ============================================================================
# 1. PROBLEMA CRÍTICO: NULL vs ZERO na DIME
# ============================================================================
print("\n" + "="*60)
print("1. PROBLEMA CRÍTICO: NULL vs ZERO (DIME)")
print("="*60)

try:
    print("\n🔍 Analisando tratamento de NULLs:")
    
    null_analysis = spark.sql("""
        SELECT 
            COUNT(*) as total,
            
            -- NULLs
            SUM(CASE WHEN VL_FATURAMENTO IS NULL THEN 1 ELSE 0 END) as fat_null,
            SUM(CASE WHEN VL_TOT_CRED IS NULL THEN 1 ELSE 0 END) as cred_null,
            SUM(CASE WHEN VL_TOT_DEB IS NULL THEN 1 ELSE 0 END) as deb_null,
            
            -- Zeros
            SUM(CASE WHEN VL_FATURAMENTO = 0 THEN 1 ELSE 0 END) as fat_zero,
            SUM(CASE WHEN VL_TOT_CRED = 0 THEN 1 ELSE 0 END) as cred_zero,
            SUM(CASE WHEN VL_TOT_DEB = 0 THEN 1 ELSE 0 END) as deb_zero,
            
            -- Critério ATUAL (ERRADO - não trata NULL)
            SUM(CASE 
                WHEN VL_FATURAMENTO = 0 THEN 1
                WHEN VL_TOT_CRED = 0 AND VL_TOT_DEB = 0 THEN 1
                ELSE 0 
            END) as criterio_atual,
            
            -- Critério CORRETO (trata NULL como zero)
            SUM(CASE 
                WHEN COALESCE(VL_FATURAMENTO, 0) = 0 THEN 1
                WHEN COALESCE(VL_TOT_CRED, 0) = 0 AND COALESCE(VL_TOT_DEB, 0) = 0 THEN 1
                ELSE 0 
            END) as criterio_corrigido,
            
            -- Casos onde NULL faz diferença
            SUM(CASE 
                WHEN (COALESCE(VL_FATURAMENTO, 0) = 0 OR 
                      (COALESCE(VL_TOT_CRED, 0) = 0 AND COALESCE(VL_TOT_DEB, 0) = 0))
                     AND NOT (VL_FATURAMENTO = 0 OR (VL_TOT_CRED = 0 AND VL_TOT_DEB = 0))
                THEN 1 ELSE 0 
            END) as diferenca_null
            
        FROM usr_sat_ods.ods_decl_dime_raw
        WHERE nu_per_ref BETWEEN 202101 AND 202509
    """).collect()[0]
    
    total = null_analysis['total']
    print(f"\n   Total de registros: {total:,}")
    print(f"\n   📊 NULLs:")
    print(f"      Faturamento NULL: {null_analysis['fat_null']:,} ({null_analysis['fat_null']/total*100:.1f}%)")
    print(f"      Crédito NULL: {null_analysis['cred_null']:,} ({null_analysis['cred_null']/total*100:.1f}%)")
    print(f"      Débito NULL: {null_analysis['deb_null']:,} ({null_analysis['deb_null']/total*100:.1f}%)")
    
    print(f"\n   📊 Zeros:")
    print(f"      Faturamento = 0: {null_analysis['fat_zero']:,} ({null_analysis['fat_zero']/total*100:.1f}%)")
    print(f"      Crédito = 0: {null_analysis['cred_zero']:,} ({null_analysis['cred_zero']/total*100:.1f}%)")
    print(f"      Débito = 0: {null_analysis['deb_zero']:,} ({null_analysis['deb_zero']/total*100:.1f}%)")
    
    print(f"\n   ⚠️ COMPARAÇÃO DE CRITÉRIOS:")
    print(f"      Critério ATUAL (sem COALESCE): {null_analysis['criterio_atual']:,} ({null_analysis['criterio_atual']/total*100:.1f}%)")
    print(f"      Critério CORRETO (com COALESCE): {null_analysis['criterio_corrigido']:,} ({null_analysis['criterio_corrigido']/total*100:.1f}%)")
    print(f"      🔴 DIFERENÇA: {null_analysis['diferenca_null']:,} registros afetados por NULLs!")
    
    if null_analysis['diferenca_null'] > 0:
        print(f"\n   ❌ PROBLEMA CONFIRMADO: O critério atual IGNORA {null_analysis['diferenca_null']:,} declarações zeradas por causa de NULLs!")
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 2. INCONSISTÊNCIA: INDÍCIOS vs FLAGS
# ============================================================================
print("\n" + "="*60)
print("2. INCONSISTÊNCIA: INDÍCIOS DA FONTE vs FLAGS NAS TABELAS")
print("="*60)

try:
    print("\n🔍 Comparando indícios originais com flags nas tabelas auxiliares:")
    
    # Indícios na fonte original
    print("\n📊 FONTE ORIGINAL (neaf.empresa_indicio):")
    indicios_fonte = spark.sql("""
        SELECT 
            cd_indicio,
            COUNT(DISTINCT nu_cpf_cnpj) as qtd_cnpjs
        FROM neaf.empresa_indicio
        WHERE cd_uf = 'SC' 
          AND cd_atual = 1
          AND cd_indicio IN (11, 12)
        GROUP BY cd_indicio
    """).toPandas()
    
    for _, row in indicios_fonte.iterrows():
        tipo = "DIME (11)" if row['cd_indicio'] == 11 else "PGDAS (12)"
        print(f"   Indício {int(row['cd_indicio'])} ({tipo}): {int(row['qtd_cnpjs']):,} CNPJs")
    
    # Flags nas tabelas auxiliares
    print("\n📊 TABELA cancel_zero_normal:")
    flags_normal = spark.sql("""
        SELECT 
            SUM(CASE WHEN flag_omissao_dime_indicio = 1 THEN 1 ELSE 0 END) as flag_dime,
            SUM(CASE WHEN flag_omissao_pgdas_indicio = 1 THEN 1 ELSE 0 END) as flag_pgdas
        FROM teste.cancel_zero_normal
    """).collect()[0]
    print(f"   flag_omissao_dime_indicio = 1: {flags_normal['flag_dime']:,}")
    print(f"   flag_omissao_pgdas_indicio = 1: {flags_normal['flag_pgdas']:,} ⚠️ (deveria ser 0)")
    
    print("\n📊 TABELA cancel_zero_simples:")
    flags_simples = spark.sql("""
        SELECT 
            SUM(CASE WHEN flag_omissao_dime_indicio = 1 THEN 1 ELSE 0 END) as flag_dime,
            SUM(CASE WHEN flag_omissao_pgdas_indicio = 1 THEN 1 ELSE 0 END) as flag_pgdas
        FROM teste.cancel_zero_simples
    """).collect()[0]
    print(f"   flag_omissao_dime_indicio = 1: {flags_simples['flag_dime']:,} ⚠️ (deveria ser 0)")
    print(f"   flag_omissao_pgdas_indicio = 1: {flags_simples['flag_pgdas']:,}")
    
    print("\n📊 TABELA credito_dime_completo:")
    flags_completo = spark.sql("""
        SELECT 
            SUM(CASE WHEN flag_omissao_dime_normal = 1 THEN 1 ELSE 0 END) as flag_dime_normal,
            SUM(CASE WHEN flag_omissao_pgdas_simples = 1 THEN 1 ELSE 0 END) as flag_pgdas_simples
        FROM teste.credito_dime_completo
    """).collect()[0]
    print(f"   flag_omissao_dime_normal = 1: {flags_completo['flag_dime_normal']:,}")
    print(f"   flag_omissao_pgdas_simples = 1: {flags_completo['flag_pgdas_simples']:,}")
    
    # Análise da discrepância
    print("\n🔴 ANÁLISE DA DISCREPÂNCIA:")
    dime_fonte = indicios_fonte[indicios_fonte['cd_indicio'] == 11]['qtd_cnpjs'].values[0] if len(indicios_fonte[indicios_fonte['cd_indicio'] == 11]) > 0 else 0
    pgdas_fonte = indicios_fonte[indicios_fonte['cd_indicio'] == 12]['qtd_cnpjs'].values[0] if len(indicios_fonte[indicios_fonte['cd_indicio'] == 12]) > 0 else 0
    
    print(f"   DIME (indício 11):")
    print(f"      Fonte original: {dime_fonte:,}")
    print(f"      cancel_zero_normal: {flags_normal['flag_dime']:,}")
    print(f"      credito_dime_completo: {flags_completo['flag_dime_normal']:,}")
    print(f"      ❌ Perda: {dime_fonte - flags_completo['flag_dime_normal']:,} CNPJs ({(dime_fonte - flags_completo['flag_dime_normal'])/dime_fonte*100:.1f}%)")
    
    print(f"\n   PGDAS (indício 12):")
    print(f"      Fonte original: {pgdas_fonte:,}")
    print(f"      cancel_zero_simples: {flags_simples['flag_pgdas']:,}")
    print(f"      credito_dime_completo: {flags_completo['flag_pgdas_simples']:,}")
    print(f"      ❌ Perda: {pgdas_fonte - flags_completo['flag_pgdas_simples']:,} CNPJs ({(pgdas_fonte - flags_completo['flag_pgdas_simples'])/pgdas_fonte*100:.1f}%)")
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 3. INVESTIGAR: PROBLEMA NO JOIN
# ============================================================================
print("\n" + "="*60)
print("3. INVESTIGAR: PERDA DE DADOS NO JOIN")
print("="*60)

try:
    print("\n🔍 Analisando CNPJs com indício 11 (DIME) que não aparecem na tabela final:")
    
    # CNPJs com indício 11 na fonte
    cnpjs_indicio11 = spark.sql("""
        SELECT COUNT(DISTINCT nu_cpf_cnpj) as qtd
        FROM neaf.empresa_indicio
        WHERE cd_uf = 'SC' 
          AND cd_atual = 1
          AND cd_indicio = 11
    """).collect()[0]['qtd']
    
    # CNPJs com flag na tabela final
    cnpjs_flag = spark.sql("""
        SELECT COUNT(*) as qtd
        FROM teste.credito_dime_completo
        WHERE flag_omissao_dime_normal = 1
    """).collect()[0]['qtd']
    
    print(f"\n   📊 CNPJs com indício 11 na fonte: {cnpjs_indicio11:,}")
    print(f"   📊 CNPJs com flag na tabela final: {cnpjs_flag:,}")
    print(f"   ❌ CNPJs perdidos: {cnpjs_indicio11 - cnpjs_flag:,} ({(cnpjs_indicio11 - cnpjs_flag)/cnpjs_indicio11*100:.1f}%)")
    
    # Verificar se o problema é no join entre neaf.empresa_indicio e credito_dime_completo
    print("\n🔍 Verificando se CNPJs com indício 11 têm crédito acumulado:")
    
    join_analysis = spark.sql("""
        WITH indicios_11 AS (
            SELECT DISTINCT 
                REGEXP_REPLACE(TRIM(CAST(nu_cpf_cnpj AS STRING)), '[^0-9]', '') AS cnpj_limpo
            FROM neaf.empresa_indicio
            WHERE cd_uf = 'SC' 
              AND cd_atual = 1
              AND cd_indicio = 11
        ),
        empresas_credito AS (
            SELECT DISTINCT 
                REGEXP_REPLACE(TRIM(CAST(nu_cnpj AS STRING)), '[^0-9]', '') AS cnpj_limpo
            FROM teste.credito_dime_completo
        )
        SELECT 
            (SELECT COUNT(*) FROM indicios_11) as total_indicios,
            (SELECT COUNT(*) FROM empresas_credito) as total_credito,
            COUNT(*) as match_count
        FROM indicios_11 i
        JOIN empresas_credito c ON i.cnpj_limpo = c.cnpj_limpo
    """).collect()[0]
    
    print(f"   Total com indício 11: {join_analysis['total_indicios']:,}")
    print(f"   Total na credito_dime_completo: {join_analysis['total_credito']:,}")
    print(f"   Matches (JOIN): {join_analysis['match_count']:,}")
    print(f"   ❌ CNPJs com indício 11 MAS SEM crédito: {join_analysis['total_indicios'] - join_analysis['match_count']:,}")
    
    print("\n💡 CONCLUSÃO:")
    if join_analysis['total_indicios'] - join_analysis['match_count'] > 0:
        print(f"   🔴 PROBLEMA IDENTIFICADO: {join_analysis['total_indicios'] - join_analysis['match_count']:,} empresas têm indício de omissão")
        print(f"      mas NÃO aparecem na credito_dime_completo porque não têm crédito acumulado!")
        print(f"      Isso significa que a tabela credito_dime_completo SÓ inclui empresas COM crédito,")
        print(f"      mas deveria incluir TODAS as empresas com indicações de fraude!")
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 4. ANÁLISE: EMPRESAS NA cancel_zero_normal MAS NÃO NA credito_dime_completo
# ============================================================================
print("\n" + "="*60)
print("4. PERDA NO JOIN: cancel_zero_normal → credito_dime_completo")
print("="*60)

try:
    print("\n🔍 Analisando perda de empresas no pipeline:")
    
    pipeline_analysis = spark.sql("""
        WITH normal_cnpjs AS (
            SELECT DISTINCT 
                REGEXP_REPLACE(TRIM(CAST(nu_cnpj AS STRING)), '[^0-9]', '') AS cnpj_limpo
            FROM teste.cancel_zero_normal
            WHERE periodos_omissos > 0 OR periodos_zerados > 0
        ),
        completo_cnpjs AS (
            SELECT DISTINCT 
                REGEXP_REPLACE(TRIM(CAST(nu_cnpj AS STRING)), '[^0-9]', '') AS cnpj_limpo
            FROM teste.credito_dime_completo
        )
        SELECT 
            (SELECT COUNT(*) FROM normal_cnpjs) as na_normal,
            (SELECT COUNT(*) FROM completo_cnpjs) as na_completo,
            (SELECT COUNT(*) FROM normal_cnpjs n WHERE EXISTS (SELECT 1 FROM completo_cnpjs c WHERE c.cnpj_limpo = n.cnpj_limpo)) as match
        FROM dual
    """).collect()[0]
    
    print(f"   CNPJs em cancel_zero_normal (com omissões/zerados): {pipeline_analysis['na_normal']:,}")
    print(f"   CNPJs em credito_dime_completo: {pipeline_analysis['na_completo']:,}")
    print(f"   Match: {pipeline_analysis['match']:,}")
    print(f"   ❌ Perdidos: {pipeline_analysis['na_normal'] - pipeline_analysis['match']:,} ({(pipeline_analysis['na_normal'] - pipeline_analysis['match'])/pipeline_analysis['na_normal']*100:.1f}%)")
    
    # Investigar POR QUE empresas são perdidas
    print("\n🔍 Motivo da perda:")
    print("   A tabela credito_dime_completo usa como base:")
    print("   'dados_filtrados' que requer: vl_cred_mes_anterior > 0 AND vl_cred_mes_seguinte > 0")
    print("   OU SEJA: SÓ empresas com CRÉDITO ACUMULADO são incluídas!")
    
except Exception as e:
    print(f"   ❌ Erro: {str(e)[:300]}")

# ============================================================================
# 5. RESUMO DOS PROBLEMAS E SOLUÇÕES
# ============================================================================
print("\n" + "="*60)
print("5. RESUMO: PROBLEMAS IDENTIFICADOS E SOLUÇÕES")
print("="*60)

print("""
🔴 PROBLEMA 1: TRATAMENTO DE NULL
   ❌ Situação atual: Critério ignora valores NULL
   ✅ Solução: Usar COALESCE(campo, 0) em todos os critérios de "zerado"
   
   SQL CORRETO:
   CASE 
       WHEN COALESCE(VL_FATURAMENTO, 0) = 0 THEN 1
       WHEN COALESCE(VL_TOT_CRED, 0) = 0 AND COALESCE(VL_TOT_DEB, 0) = 0 THEN 1
       ELSE 0 
   END AS declaracao_zerada

🔴 PROBLEMA 2: PERDA DE EMPRESAS NO JOIN
   ❌ Situação atual: credito_dime_completo só inclui empresas COM crédito
   ✅ Solução: Mudar a base da query para incluir TODAS as empresas do cadastro,
              não só as que têm crédito acumulado
   
   SQL CORRETO:
   - Base deve ser: usr_sat_ods.vw_cad_contrib (cadastro completo)
   - Fazer LEFT JOIN com dados de crédito
   - Fazer LEFT JOIN com dados de enriquecimento

🔴 PROBLEMA 3: INCONSISTÊNCIA DE FLAGS
   ❌ Situação atual: Indícios da fonte (13,160 DIME, 6,005 PGDAS) ≠ Flags finais (3,430 DIME, 82 PGDAS)
   ✅ Solução: Após corrigir Problema 2, as flags devem bater com os indícios da fonte

🔴 PROBLEMA 4: FLAGS COM NOMENCLATURA ERRADA
   ❌ Situação atual: flag_omissao_dime_simples e flag_omissao_pgdas_normal existem (impossível!)
   ✅ Solução: Remover essas colunas e manter apenas:
              - flag_omissao_dime_normal (ou flag_omissao_dime)
              - flag_omissao_pgdas_simples (ou flag_omissao_pgdas)

💡 AÇÃO IMEDIATA:
   1. Corrigir SQL das tabelas auxiliares (cancel_zero_*) com COALESCE
   2. Corrigir SQL da credito_dime_completo mudando a base para o cadastro completo
   3. Remover colunas de flags incorretas
   4. Re-executar toda a pipeline
   5. Validar se os números batem com a fonte original
""")

print("\n" + "="*80)
print("FIM DA INVESTIGAÇÃO")
print("="*80)

INVESTIGAÇÃO: PROBLEMAS CRÍTICOS IDENTIFICADOS

1. PROBLEMA CRÍTICO: NULL vs ZERO (DIME)

🔍 Analisando tratamento de NULLs:

   Total de registros: 4,428,586

   📊 NULLs:
      Faturamento NULL: 1,633,194 (36.9%)
      Crédito NULL: 1,994,781 (45.0%)
      Débito NULL: 2,185,371 (49.3%)

   📊 Zeros:
      Faturamento = 0: 457,812 (10.3%)
      Crédito = 0: 0 (0.0%)
      Débito = 0: 0 (0.0%)

   ⚠️ COMPARAÇÃO DE CRITÉRIOS:
      Critério ATUAL (sem COALESCE): 457,812 (10.3%)
      Critério CORRETO (com COALESCE): 2,207,414 (49.8%)
      🔴 DIFERENÇA: 0 registros afetados por NULLs!

2. INCONSISTÊNCIA: INDÍCIOS DA FONTE vs FLAGS NAS TABELAS

🔍 Comparando indícios originais com flags nas tabelas auxiliares:

📊 FONTE ORIGINAL (neaf.empresa_indicio):
   Indício 12 (PGDAS (12)): 6,005 CNPJs
   Indício 11 (DIME (11)): 13,160 CNPJs

📊 TABELA cancel_zero_normal:
   flag_omissao_dime_indicio = 1: 12,497
   flag_omissao_pgdas_indicio = 1: 675 ⚠️ (deveria ser 0)

📊 TABELA cancel_zero_simples:
   f


   📊 CNPJs com indício 11 na fonte: 13,160
   📊 CNPJs com flag na tabela final: 3,430
   ❌ CNPJs perdidos: 9,730 (73.9%)

🔍 Verificando se CNPJs com indício 11 têm crédito acumulado:


   Total com indício 11: 13,160
   Total na credito_dime_completo: 54,777
   Matches (JOIN): 3,413
   ❌ CNPJs com indício 11 MAS SEM crédito: 9,747

💡 CONCLUSÃO:
   🔴 PROBLEMA IDENTIFICADO: 9,747 empresas têm indício de omissão
      mas NÃO aparecem na credito_dime_completo porque não têm crédito acumulado!
      Isso significa que a tabela credito_dime_completo SÓ inclui empresas COM crédito,
      mas deveria incluir TODAS as empresas com indicações de fraude!

4. PERDA NO JOIN: cancel_zero_normal → credito_dime_completo

🔍 Analisando perda de empresas no pipeline:
   ❌ Erro: [TABLE_OR_VIEW_NOT_FOUND] The table or view `dual` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop

5. RESUMO: PROBLEMAS IDENTIFICADOS E SOLUÇÕES

🔴 PROBLEMA 1: TRATAMENTO DE NULL
   ❌ Situação atual: Critério ig